## TODO

Need to incorporate player offensive rating and team defensive rating somehow. Potential uses could be adding/subtracting to the montecarlo mean, or even using it to scale it.

In [133]:
import pandas as pd
import urllib
import bs4 as bs
import numpy as np

In [134]:
team_conferences = {
    'Atlanta Hawks': 'Eastern',
    'Boston Celtics': 'Eastern',
    'Brooklyn Nets': 'Eastern',
    'Charlotte Hornets': 'Eastern',
    'Chicago Bulls': 'Eastern',
    'Cleveland Cavaliers': 'Eastern',
    'Detroit Pistons': 'Eastern',
    'Indiana Pacers': 'Eastern',
    'Miami Heat': 'Eastern',
    'Milwaukee Bucks': 'Eastern',
    'New York Knicks': 'Eastern',
    'Orlando Magic': 'Eastern',
    'Philadelphia 76ers': 'Eastern',
    'Toronto Raptors': 'Eastern',
    'Washington Wizards': 'Eastern',
    'Dallas Mavericks': 'Western',
    'Denver Nuggets': 'Western',
    'Golden State Warriors': 'Western',
    'Houston Rockets': 'Western',
    'Los Angeles Clippers': 'Western',
    'Los Angeles Lakers': 'Western',
    'Memphis Grizzlies': 'Western',
    'Minnesota Timberwolves': 'Western',
    'New Orleans Pelicans': 'Western',
    'Oklahoma City Thunder': 'Western',
    'Phoenix Suns': 'Western',
    'Portland Trail Blazers': 'Western',
    'Sacramento Kings': 'Western',
    'San Antonio Spurs': 'Western',
    'Utah Jazz': 'Western'
}

In [135]:
# Parameters

url = "https://www.teamrankings.com/nba/player/nikola-jokic/game-log"

opponent = "Los Angeles Lakers"
points_line = 27
points_over = -113
points_under = -113

show_all = True # Show whether to print all odds or just the favourable ones


In [136]:
# Retrieve data

source = urllib.request.urlopen(url).read()
soup = bs.BeautifulSoup(source,'lxml')
table = str(soup.find_all('table')[0])
data = pd.read_html(table)[0]


In [137]:
# Preprocessing

data = data.drop(index=data.index[-2:]) # Remove last two rows as they're totals

# Turn minutes into a number of minutes
def time_to_minutes(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes + seconds / 60
data["Min"] = data["Min"].apply(time_to_minutes)

# Add team's conference to data
data["Conference"] = data["Opp"].map(team_conferences) # 

In [138]:
# Calculate Projection

averages = [] # Create list of averages to find mean over

# Find way to determine if home or away game?

averages.append(((data["PTS"]/data["Min"]) * data["Min"]).mean()) # get the average of average points per minute multiplied by average minutes
averages.append(data[:-1]["PTS"].iloc[0]) # previous game
averages.append(data[-5:]["PTS"].mean()) # last 5 games
averages.append(data[-10:]["PTS"].mean()) # last 10 games 
if not pd.isna(data[data["Opp"] == opponent]["PTS"].mean()): # games where they played the opponent, check to make sure one exists
    averages.append(data[data["Opp"] == opponent]["PTS"].mean())
if not pd.isna(data[data["Conference"] == team_conferences[opponent]]["PTS"].mean()): # games where they played the conference, check to make sure one exists
    averages.append(data[data["Conference"] == team_conferences[opponent]]["PTS"].mean())

np_averages = np.array(averages)
projected_average = np_averages.mean() # get average of accumulated values

In [139]:
# Run Montecarlo 

std = np.array(data["PTS"]).std()  # calculate standard deviation from point history
n = 100000

samples = np.random.normal(projected_average, std, n)

# Counting how many samples are above the check_value
count_above = np.sum(samples > points_line)

percentage_over = count_above/n
percentage_under = (n - count_above)/n

In [146]:
# Calculate deviated projection as combination of calculated projection and montecarlo

def vegas_to_probability(vegas_odds):
    
    if vegas_odds > 0:
        decimal_odds = (vegas_odds / 100) + 1
    else:
        decimal_odds = (100 / abs(vegas_odds)) + 1

    return (1 / decimal_odds)

# Iterate over odds?
if vegas_to_probability(points_over) > points_over:
    print()


0.5305164319248826